In [1]:
import base64
import json
import os
from base64 import b64encode,b64decode

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

In [3]:
import base64
import json
import os
import threading
from base64 import b64encode,b64decode

from prompt_toolkit import prompt
from prompt_toolkit.filters import Condition
from prompt_toolkit.key_binding import KeyBindings

from prompt_toolkit.validation import Validator
from prompt_toolkit.completion import PathCompleter
 
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.fernet import Fernet, InvalidToken


In [5]:
# Returns a string representation of a salt
def generate_salt(size=16):
    return b64encode(os.urandom(size)).decode()

def create_key(pass_phrase, salt=generate_salt()):
    salt =  b64decode(salt)
    # Taken from the website itse
    # https://cryptography.io/en/latest/fernet/#using-passwords-with-fernet
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=32,
        salt=salt,
        iterations=10000
    )

    key = base64.urlsafe_b64encode(kdf.derive(pass_phrase.encode()))

    return key  


def encrypt(data, pass_phrase, salt=generate_salt()):
    pass


class Crypto:

    
    def __init__(self, pass_phrase, data, salt=generate_salt()):
        """initializes the crypto class

        Args:
            pass_phrase (str): the password for encrypting or decrypting
            data (str): the data we want to decrypt or encrypt
            salt (str, optional): string representation of a salt we want to use for creating key. Defaults to generate_salt().
        """
        self.data = str(data)
        self.salt = salt
        self.pass_phrase = pass_phrase
        
        # self.encrypt_status = False

        self.key = self.create_key()


    def create_key(self):
        """[summary]

        Returns:
            [type]: [description]
        """
        
        # Taken from the website itse
        # https://cryptography.io/en/latest/fernet/#using-passwords-with-fernet
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=b64decode(self.salt),
            iterations=10000
        )

        key = base64.urlsafe_b64encode(kdf.derive(self.pass_phrase.encode()))

        return key    

    def encrypt(self):
        # if self.encrypt_status == True:
        #     return

        f = Fernet(self.key)

        self.data = f.encrypt(self.data.encode()).decode()
        return self

        # self.encrypt_status = True
    def decrypt(self):
        """[summary]

        Returns:
            [type]: [description]
        """
        # if self.encrypt_status == False:
        #     return

        f = Fernet(self.key)

        self.data = f.decrypt(self.data.encode()).decode()
        # self.encrypt_status = False
        return self

    def __str__(self):
        return json.dumps(self.to_dict())

    def to_dict(self):
        return {
            'salt':self.salt,
            'data':self.data
        }
  

In [6]:
passwd = "MYpass"

In [54]:
c.encrypt()
ssc = str(c)

In [57]:
str(c)

'{"salt": "sajQTxfQPrr+CtGHKqNIOA==", "data": "gAAAAABhGkYvBbNXLayi3T-om9xhQVKdHz2bw3N3iDQi3UvRQy1AOrsJXvaY2_8sfKvvldpVJmYFJLD8IAUjZS7TNzH8WRMOww=="}'

In [58]:
s = json.loads(ssc)
cc = crypto("ABCD", **s)
cc.decrypt()
cc.data

'SECRET MESSAGE'

In [22]:
mydict

{'user1': '{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_uv7Ua_mSd86AIw51y4f0MVcDa-ySo4oEfSQXnYqDzGyefkxLIB-MU_KkOd6MTWkOtKNlnWC-7I2kILjaFOMnEw=="}',
 'user2': '{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_S97ArIQHg9ZqJPM_3rqrxst949JqtL-LqOO5M9J1GlHHwNhW8zOrgGXmHbU0EYxvraouk2hTbTtEbyF40sJTXQ=="}',
 'user3': '{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_gSru1Jpezdy3bALWjCmhtAldbpvdZHxY-tAEouRauH3HLuFt5A7ob3xLMj5HP7h0c5RFhkq8XF567tYJC56btA=="}'}

In [8]:
passwd = "MYpass"

mydict={
    'user1': str(Crypto(passwd, "PASS1").encrypt()),
    'user2': str(Crypto(passwd, "PASS2").encrypt()),
    'user3': str(Crypto(passwd, "PASS3").encrypt())
}

In [9]:
mydict['user1']

'{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_uv7Ua_mSd86AIw51y4f0MVcDa-ySo4oEfSQXnYqDzGyefkxLIB-MU_KkOd6MTWkOtKNlnWC-7I2kILjaFOMnEw=="}'

In [10]:
Crypto(passwd, **json.loads(mydict['user2'])).decrypt().data

'PASS2'

In [11]:
lastf = Crypto(passwd, json.dumps(mydict)).encrypt()
str(lastf)

'{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1_EGuJ-3aBHuAi1TDzhH9lplWOvq27xdmNN3cKAJ0OaKjAhpb1cOqLlj33obUupe5urztBnpsaTmti-9pLtI_hZt2fSm7jrO_qjbQo_itMPXryts8SKM1rk8necYcZkUfPP4uLAEGXUZz6lj18OeX0ueO8J9iwNGtTaJ1KZTP3TavZfv6Fcv39SSUshjpvagsERx5--oN839sYAPbn7HzzXlWiC7yy2hsLlctBV0OjAmiAb5Fm-EsIVVTY2KxmllwSCzMBuS-69Fc_zn6cLjeDlYrSBjNPe5DxJMaZ9HFaiMyPEi6nTrIYFuGAGnjVGfEkgeGGZzA-8Ph_86JtX99b7SklcLp6_mcdNdpwdTy5mggrMMeTBWAtUouW8XaQUwEOAek255tQJfMzyc3EaxH9pKriFp-G8NsgjzFRKfwEsc-WN2tcxiDo0CQGNfdczGwu5jGEVSTTNnXe9aTALO-TWPH2M-ouNdnIHO3jR3h7Rcx3DIFq1fPk56SjDI65H2Vs5k2NeqolmSB732z6tqqy9VtBun-cb9Sa8CtEaooe6E1rSOjbhOq7Bt2BVBG6C8jCm51xd4ciVOlFPXhn2Xklnlap7ccng7eUuODxFGEn02eHTUPPFp0SgdM9dOwwdIBL7kp4nZzyd9v0g9TD0t-9T4WZ4kz1D9C3G_ggWPySJiJwb2OcYLtyQ6iFUvqstCLSuedP8sSMdZOeDE-krzrsvqi6HsDPKdTk8WeRuDZFzxDM="}'

In [25]:
with open("vault.json", "w") as f:
    f.write(str(lastf))

In [12]:
firstf = Crypto(passwd, **json.loads(str(lastf))).decrypt()
(firstf.to_dict())

{'salt': 'TRyAoec+UMMyIwWZe9pGFA==',
 'data': '{"user1": "{\\"salt\\": \\"TRyAoec+UMMyIwWZe9pGFA==\\", \\"data\\": \\"gAAAAABhG1-_uv7Ua_mSd86AIw51y4f0MVcDa-ySo4oEfSQXnYqDzGyefkxLIB-MU_KkOd6MTWkOtKNlnWC-7I2kILjaFOMnEw==\\"}", "user2": "{\\"salt\\": \\"TRyAoec+UMMyIwWZe9pGFA==\\", \\"data\\": \\"gAAAAABhG1-_S97ArIQHg9ZqJPM_3rqrxst949JqtL-LqOO5M9J1GlHHwNhW8zOrgGXmHbU0EYxvraouk2hTbTtEbyF40sJTXQ==\\"}", "user3": "{\\"salt\\": \\"TRyAoec+UMMyIwWZe9pGFA==\\", \\"data\\": \\"gAAAAABhG1-_gSru1Jpezdy3bALWjCmhtAldbpvdZHxY-tAEouRauH3HLuFt5A7ob3xLMj5HP7h0c5RFhkq8XF567tYJC56btA==\\"}"}'}

In [18]:
ag = json.loads(firstf.to_dict()['data'])
ag

{'user1': '{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_uv7Ua_mSd86AIw51y4f0MVcDa-ySo4oEfSQXnYqDzGyefkxLIB-MU_KkOd6MTWkOtKNlnWC-7I2kILjaFOMnEw=="}',
 'user2': '{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_S97ArIQHg9ZqJPM_3rqrxst949JqtL-LqOO5M9J1GlHHwNhW8zOrgGXmHbU0EYxvraouk2hTbTtEbyF40sJTXQ=="}',
 'user3': '{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_gSru1Jpezdy3bALWjCmhtAldbpvdZHxY-tAEouRauH3HLuFt5A7ob3xLMj5HP7h0c5RFhkq8XF567tYJC56btA=="}'}

In [21]:
ag['user3']

'{"salt": "TRyAoec+UMMyIwWZe9pGFA==", "data": "gAAAAABhG1-_gSru1Jpezdy3bALWjCmhtAldbpvdZHxY-tAEouRauH3HLuFt5A7ob3xLMj5HP7h0c5RFhkq8XF567tYJC56btA=="}'

In [34]:
the_dict = {
    "google": {
        "mathurshiva901": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        },
        "mathurshiva90b": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        },
        "mathurshiva90c": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        },
        "mathurshiva90d": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        }
    },  
    "orkut": {
        "mathurshiva901": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        },
        "mathurshiva90b": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        },
        "mathurshiva90c": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        },
        "mathurshiva90d": {
            "password": {
                "pass": "pass_encrypted",
                "salt": "unique_salt"
            }
        }
    }
}

dictt = {}
for keys in the_dict:
    dictt[keys] = {}
    for keys2 in the_dict[keys]:
        dictt[keys][keys2] = {}
        for keys3 in the_dict[keys][keys2]:
            dictt[keys][keys2][keys3] = None

            
from prompt_toolkit.completion import NestedCompleter

out = prompt(">", completer= NestedCompleter.from_nested_dict(dictt))
out

dict_keys(['google', 'orkut'])

In [39]:
dictt

{'google': {'mathurshiva901': {'password': None},
  'mathurshiva90b': {'password': None},
  'mathurshiva90c': {'password': None},
  'mathurshiva90d': {'password': None}},
 'orkut': {'mathurshiva901': {'password': None},
  'mathurshiva90b': {'password': None},
  'mathurshiva90c': {'password': None},
  'mathurshiva90d': {'password': None}}}

NoConsoleScreenBufferError: Found xterm-color, while expecting a Windows console. Maybe try to run this program using "winpty" or run it in cmd.exe instead. Or otherwise, in case of Cygwin, use the Python executable that is compiled for Cygwin.